# DataQuest Guided Project: Designing and Creating a Database

## Major League Baseball

The goal of this project is to create a normalized database from a CSV source file containing 161 datapoints on 171,907 major league baseball games dating back to 1874. The final database will allow users to easily analyze and maintain using standard SQL.

#### Data

The source of this data set is [Retrosheet](https://www.retrosheet.org/gamelogs/index.html), which has compiled facts on baseball games ranging back to the 1800's. The main CSV file, game_log.csv, was created by combining 127 separate CSV files and pre-cleaned by Dataquest to remove some inconsistancies. In addition to the game log, three other helper files have been sourced from Retrosheet to help build fact tables on parks, people, and teams.

Additionally, Retrosheet has provided a text file explaining the fields located in the game_log.csv file as well as this disclaimer:

- The information used here was obtained free of charge from and is copyrighted by Retrosheet. Interested parties may contact Retrosheet at "www.retrosheet.org". 

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
#adjust pandas output options to avoid truncated displays
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

## Import game log and supporting docs

In [3]:
log = pd.read_csv('game_log.csv', low_memory=False)
park = pd.read_csv('park_codes.csv')
person = pd.read_csv('person_codes.csv')
team = pd.read_csv('team_codes.csv')

In [4]:
#Definitions
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


## Exploratory Data Analysis / Dataset Overview

#### Park Codes

The park_codes table contains 252 records of ballparks along with 9 columns of details including a park_id which serves as a primary key. Details include the name, city, and state of the park as well as the open and closing dates. Ballpark details go back to 1874.

The primary key field, park_id, ties back to the same field in the game_log. Example value: **'BOS07'** for Fenway Park in Boston.

This table could also be used to identify which league played in the park, including defunct leagues. The following values indicate league;

- 'NL' - National League (Modern)
- AL - American League (Modern)
- AA - American Association (1882-1891)
- UA - Union Association (1 season in 1884)
- FL - Federal League (1913-1915)
- PL - Player's League (1890)



In [5]:
print(park.shape)
print(park.columns)
print(park.head())
print(park.tail())
print(park['start'].max())
print(park['league'].value_counts())

(252, 9)
Index(['park_id', 'name', 'aka', 'city', 'state', 'start', 'end', 'league',
       'notes'],
      dtype='object')
  park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/188

#### Person Codes

The person_codes table contains 7 columns of data on 20,494 people, including players, managers, and umpires. Information includes an 'id' field as the primary key which aligns to numerous game_log columns which reference player ids assiciated with the plays being recorded. This also could include manager and umpire information. Field names often include an 'h' or 'v' in the column name to indicate home or away team. Examples of these fields include;
- hp_umpire_id
- 1b_umpire_id
- 2b_umpire_id
- v_manager_id
- h_manager_id
- winning_pitcher_id
- losing_pitcher_id
- saving_pitcher_id
- winning_rbi_batter_id
- v_starting_pitcher_id
- v_player_1_id
- v_player_2_id

It is worth noting that a person's role in a game, including their role as a player, umpire, or manager can change over the course of their career. A player's position can change over the course of a game, so this would not be hard coded into their person entry.

In [6]:
print(person.shape)
print(person.columns)
print(person.head())
print(person.tail())

(20494, 7)
Index(['id', 'last', 'first', 'player_debut', 'mgr_debut', 'coach_debut',
       'ump_debut'],
      dtype='object')
         id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN
             id      last   first player_debut mgr_debut coach_debut ump_debut
20489  zuvep001   Zuvella    Paul   09/04/1982       NaN  04/02/1996       NaN
20490  zuveg101  Zuverink  George   04/21/1951       NaN         NaN       NaN
20491  zwild101  Zwilling   Dutch   08/14/1910       NaN  04/15/1941       NaN
20492  zycht001      Zych    Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson     NaN

#### Team Codes

The team_codes dataset includes 150 rows of information about teams, including their league, city, start/end dates and nicknames. Additionally, there is a franchise identifier and a sequence column that can be used to track changes to the teams over time. For example, the Washington Senators existed in the AA league in 1891 (seq=1) and then changed to the National League in 1892 (seq=2).

The team_id field is the primary key and it corresponds to the game_log table in the fields, 'v_name' and 'h_name'

In [7]:
print(team.shape)
print(team.columns)
print(team.head())
print(team.tail())
print()
print(team[team['franch_id'] == 'WS9'])

(150, 8)
Index(['team_id', 'league', 'start', 'end', 'city', 'nickname', 'franch_id',
       'seq'],
      dtype='object')
  team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1
    team_id league  start   end        city   nickname franch_id  seq
145     WS8     NL   1886  1889  Washington   Senators       WS8    1
146     WS9     AA   1891  1891  Washington   Senators       WS9    1
147     WSN     NL   1892  1899  Washington   Senators       WS9    2
148     WSU     UA   1884  1884  Washington  Nationals       WSU    1
149     MIA     NL   2012     0       Miami    Marlins       FLO    2

    team_id league

### Game Log

#### Primary Key

The game log does not contain a unique field that could be used as a primary key for the entire table, however; a combined key can be created using information from three other fields. This is consistant with Retrosheets' approach and will be helpful if new data is added later. Explaination from Retrosheets;

_**id**: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file._


#### Player Positions

In the game log, plays are designated by players involved and the positions that they are playing at the time. The defensive positions are coded using a [standard notation](https://en.wikipedia.org/wiki/Baseball_positions) ranging from 1-10 where 10 is the designated hitter. For quick reference:
 1. Pitcher
 2. Catcher
 3. 1st Base
 4. 2nd Base
 5. 3rd Base
 6. Short Stop
 7. Left Field
 8. Center Field
 9. Right Field

## DB and SQLite3 helper functions



In [8]:
db = 'mlb.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        #adjust default settings to enforce foreign keys and execute immediately.
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)


### Housekeeping: Delete any tables previously loaded to state.


In [9]:
show_tables()

previous_load_tables = ['dim_person',
                        'dim_league',
                        'dim_team',
                        'dim_park',
                        'dim_game',
                        'appearance_type',
                        'team_appearance',
                        'person_appearance']

with sqlite3.connect(db) as conn:
    for table in previous_load_tables:
        conn.execute('DROP TABLE IF EXISTS {};'.format(table))


## Load existing dataframes into DB

Create initial tables using df.to_sql() method and dictionary of table names. Drop tables if exist prior to creating in case cell gets run multiple times.

In [10]:
tables = {"person": person,
          "park": park,
          "team": team,
          "log": log}

with sqlite3.connect(db) as conn: 
    for name, data in tables.items():
        conn.execute('DROP TABLE IF EXISTS {};'.format(name))
        data.to_sql(name, conn, index=False)
    


In [11]:
show_tables()

,name,type
0,person,table
1,park,table
2,team,table
3,log,table


## Create the unique identifier field for games.

In [12]:
#add blank column
c1 = """
ALTER TABLE log
ADD COLUMN game_id TEXT;

"""

try:
    run_command(c1)
except:
    print('column already exists')
    pass

#populate field with id

c2 = """
UPDATE log
SET game_id = h_name || date || number_of_game
WHERE game_id IS NULL;

"""

run_command(c2)

In [13]:
test = """
SELECT * FROM LOG LIMIT 10;
"""
run_query(test)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,18,54.0,D,None,None,None,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,None,5.0,3.0,None,-1.0,None,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,None,2.0,1.0,None,-1.0,None,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,None,None,None,None,None,None,None,None,None,None,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,None,None,None,None,spala101,Al

## Schema design

The main goal in designing this schema will be to reduce the redundancies found in the game_log file by separating it into tables which each have a single purpose and contain only fields which are dependent on the primary key of the table. For example, the game_log has stored the names of people repeatedly along with their person_ids. We can reduce this redundancy by adding a person table to the db which contains person_id as a primary key and includes basic information like their name. This same idea can then be applied to other entities such as parks, leagues, teams, and games.

Once these basic dimension tables are created, we can start to build tables around facts, or things that happened in a particular situation, such as a team appearing in a game (how many points did they score? how many bases did they steal?) or a player appearing in a game (what was their role in the game?). The basic dimension tables can then serve as foreign keys in these fact tables and can be combined to create their primary keys. For example, the primary key to the team_appearance table would be the combination of team_id and game_id, which are the primary_keys to the team and game tables.

#### Dimension tables:
1. **dim_person** - dimensional data on people including an id (key) and name. This table omits the starting dates for various stages in their career listed in the original player dataset because this data can be derived from the player_appearance facts table. Coaching information is not included in the game_log, so it will be omitted from the dim_person table.

2. **dim_park** - dimensional data on ballparks including park_id (key), name, city, state, nickname, and other notes.

3. **dim_league** - dimensional data on league including league_id (key) and name.

4. **dim_game** - dimensional meta-data on games including date, length of game in minutes/outs, attendance, etc. Anything referring to a specific team will be held in the team_appearance fact table instead. Day (of the week) will be removed as it can be calculated from date. Finally, the day_night field has been changed to a bool value for day.

5. **dim_appearance_type** - dimensional data defining the types of appearances a person can make in a game. This will be derived from an external csv of mappings, appearance_type.csv, provided by Dataquest.

6. **dim_team** - dimensional data on teams, including league_id, city, state, nickname, and franchise.

#### Fact tables:

1. **team_appearance** - contains team level statistics for each game appearance made. Primary key will be (team_id, game_id). 

2. **person_appearance** - contains player level appearance data, including the game_id, which team they were on, and the id of the appearance type that they made. Primary key will be appearance_id, which needs to be created using person_id and game_id.


#### Schema chart

#### TODO: link schema chart.

## Create and populate tables

### Add dim_person table

In [14]:
c3 = """
CREATE TABLE IF NOT EXISTS dim_person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
"""

c4 = """
INSERT OR IGNORE INTO dim_person
    SELECT
        id AS person_id,
        first AS first_name,
        last AS last_name
    FROM
        person
;

"""

q1 = """
SELECT * FROM dim_person LIMIT 5;

"""

run_command(c3)
run_command(c4)
run_query(q1)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


### Add dim_park table

In [15]:
c5 = """
CREATE TABLE IF NOT EXISTS dim_park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
    );
"""

c6 = """
INSERT OR IGNORE INTO dim_park
    SELECT
        park_id,
        name,
        aka AS nickname,
        city,
        state,
        notes
    FROM park
;
"""

q2 = """
    SELECT * FROM dim_park LIMIT 5;
"""

run_command(c5)
run_command(c6)
run_query(q2)


,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


### Add dim_league table

In [16]:
c7 = """
CREATE TABLE IF NOT EXISTS dim_league (
    league_id TEXT PRIMARY KEY,
    name TEXT
);
"""

c8 = """
INSERT OR IGNORE INTO dim_league
    VALUES
        ("NL", "National League"),
        ("AL", "American League"),
        ("AA", "American Association"),
        ("FL", "Federal League"),
        ("PL", "Players League"),
        ("UA", "Union Association");
"""

q3 = """
    SELECT * FROM dim_league;
"""

run_command(c7)
run_command(c8)
run_query(q3)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


### Add dim_appearance_type table

In [17]:
run_command("DROP TABLE IF EXISTS appearance_type;")

appearance_type = pd.read_csv('appearance_type.csv')

c9 = """
    CREATE TABLE IF NOT EXISTS appearance_type (
        appearance_type_id TEXT PRIMARY KEY,
        name TEXT,
        category TEXT
    );
"""

run_command(c9)

with sqlite3.connect(db) as conn:
    appearance_type.to_sql('appearance_type', 
                           conn, 
                           index=False,
                           if_exists='append'
                          )
    
q4 = """
    SELECT * FROM appearance_type limit 5;
"""

run_query(q4)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


### Add dim_team table

In [18]:
c10 = """
CREATE TABLE IF NOT EXISTS dim_team (
    team_id TEXT PRIMARY KEY,
    nickname TEXT,
    city TEXT,
    league_id TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES dim_league(league_id)
);
"""

c11 = """
INSERT OR IGNORE INTO dim_team
    SELECT 
        team_id,
        nickname,
        city,
        league,
        franch_id
    FROM team;
"""

q5 = """
    SELECT * FROM dim_team LIMIT 5;
"""

run_command(c10)
run_command(c11)
run_query(q5)

,team_id,nickname,city,league_id,franch_id
0,ALT,Mountain Cities,Altoona,UA,ALT
1,ARI,Diamondbacks,Arizona,NL,ARI
2,BFN,Bisons,Buffalo,NL,BFN
3,BFP,Bisons,Buffalo,PL,BFP
4,BL1,Canaries,Baltimore,None,BL1


### Add dim_game table

In [19]:
c12 = """
CREATE TABLE IF NOT EXISTS dim_game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES dim_park(park_id)
);
"""

c13 = """
    INSERT OR IGNORE INTO dim_game
        SELECT
            game_id,
            date,
            number_of_game,
            park_id,
            length_outs,
            CASE
                WHEN day_night = "D" THEN "1"
                WHEN day_night = "N" THEN "0"
                ELSE NULL
            END AS day,
            completion,
            forefeit,
            protest,
            attendance,
            length_minutes,
            additional_info,
            acquisition_info
        FROM log
            
"""

q6 = """
    SELECT * FROM dim_game LIMIT 5;
"""

run_command(c12)
run_command(c13)
run_query(q6)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


### Add team_appearance table

Note that this table combines the previously separate and redundant home and visitor team statistics. A new column, 'home' BOOLEAN, will now indicate if the team was the home or away team for that game. As an example of the change; the columns 'v_hits' and h_hits' will be combined (UNION) into 'hits' and saved as two rows, one for the home team and one for the visitor.

In [20]:
c0 = "DROP TABLE IF EXISTS team_appearance;"
run_command(c0)

c14 = """
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES dim_team(team_id),
    FOREIGN KEY (game_id) REFERENCES dim_game(game_id)
);
"""

c15 = """
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays   
    FROM log
    
UNION
    
    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM log;
    
"""

q7 = 'SELECT * FROM team_appearance LIMIT 10;'

run_command(c14)
run_command(c15)
run_query(q7)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,ALT,ALT188405120,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,ALT,ALT188405140,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,ALT,ALT188405150,1,UA,7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,ALT,ALT188405160,1,UA,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,ALT,ALT188405170,1,UA,8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Add person_appearance table

This table needs to be constructed in a similar way to the team_appearance table, meaning that we'll need to use unions to combine separate columns into one, except that instead of only having two variations (home and away), we have 36 columns for players (2 teams * 9 positions * 2 home/away) and another set of columns for umpire and manager appearances.

Rather than typing all of the possible queries, I'll be using pythong loops and string formating to assist. The appearance_type_id table will be used as a reference and the primary key will be auto-incremented.

In [21]:
c0 = "DROP TABLE IF EXISTS person_appearance"
run_command(c0)

#create table
c16 = """
CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    FOREIGN KEY (person_id) REFERENCES dim_person(person_id),
    FOREIGN KEY (team_id) REFERENCES dim_team(team_id),
    FOREIGN KEY (game_id) REFERENCES dim_game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

run_command(c16)
run_query('SELECT * FROM person_appearance;')

,appearance_id,person_id,team_id,game_id,appearance_type_id


#### Add offensive/defensive player appearance data

In [22]:
player_template = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM log
    WHERE {hv}_player_{num}_id is not null
    
    UNION
    
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
        FROM log
        WHERE {hv}_player_{num}_id IS NOT NULL;      
"""

for hv in ['h', 'v']:
    for num in range(1, 10):
        query_vals = {"hv": hv,
                      "num": num}
        run_command(player_template.format(**query_vals))

q8 = """SELECT * FROM person_appearance LIMIT 10;"""
run_query(q8)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,erstd001,ANA,ANA200004030,D7
1,2,erstd001,ANA,ANA200004030,O1
2,3,erstd001,ANA,ANA200004040,D7
3,4,erstd001,ANA,ANA200004040,O1
4,5,erstd001,ANA,ANA200004050,D7
5,6,erstd001,ANA,ANA200004050,O1
6,7,erstd001,ANA,ANA200004070,D7
7,8,erstd001,ANA,ANA200004070,O1
8,9,erstd001,ANA,ANA200004080,D7
9,10,erstd001,ANA,ANA200004080,O1


#### Add umpire, manager, and awards data

In [23]:
q17 = """
    INSERT OR IGNORE INTO person_appearance (
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
    
        SELECT
            game_id,
            NULL,
            hp_umpire_id,
            "UHP"
        FROM log
        WHERE hp_umpire_id IS NOT NULL
    
    UNION
    
        SELECT
            game_id,
            NULL,
            [1b_umpire_id],
            "U1B"
        FROM log
        WHERE [1b_umpire_id] IS NOT NULL
        
    UNION
    
        SELECT
            game_id,
            NULL,
            [2b_umpire_id],
            "U2B"
        FROM log
        WHERE [2b_umpire_id] IS NOT NULL
        
    UNION
    
        SELECT
            game_id,
            NULL,
            [3b_umpire_id],
            "U3B"
        FROM log
        WHERE [3b_umpire_id] IS NOT NULL

    UNION
    
        SELECT
            game_id,
            NULL,
            lf_umpire_id,
            "ULF"
        FROM log
        WHERE lf_umpire_id IS NOT NULL

    UNION
    
        SELECT
            game_id,
            NULL,
            rf_umpire_id,
            "URF"
        FROM log
        WHERE rf_umpire_id IS NOT NULL
        
    UNION
    
        SELECT
            game_id,
            v_name,
            v_manager_id,
            "MM"
        FROM log
        WHERE v_manager_id IS NOT NULL
        
    UNION
    
        SELECT
            game_id,
            h_name,
            h_manager_id,
            "MM"
        FROM log
        WHERE h_manager_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            winning_pitcher_id,
            "AWP"
        FROM log
        WHERE winning_pitcher_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            losing_pitcher_id,
            "ALP"
        FROM log
        WHERE losing_pitcher_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            saving_pitcher_id,
            "ASP"
        FROM log
        WHERE saving_pitcher_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            saving_pitcher_id,
            "ASP"
        FROM log
        WHERE saving_pitcher_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            winning_rbi_batter_id,
            "AWB"
        FROM log
        WHERE winning_rbi_batter_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            v_name,
            v_starting_pitcher_id,
            "PSP"
        FROM log
        WHERE v_starting_pitcher_id IS NOT NULL
        
    UNION

      SELECT
            game_id,
            h_name,
            h_starting_pitcher_id,
            "PSP"
        FROM log
        WHERE h_starting_pitcher_id IS NOT NULL

"""


run_command(q17)

In [24]:
#check values
q9 = """
SELECT appearance_type_id, count(person_id), count(team_id), count(game_id) 
FROM person_appearance 
group by appearance_type_id
order by 1;
"""
run_query(q9)

,appearance_type_id,count(person_id),count(team_id),count(game_id)
0,ALP,140229,140229,140229
1,ASP,48018,48018,48018
2,AWB,105699,105699,105699
3,AWP,140229,140229,140229
4,D1,220584,220584,220584
5,D10,61092,61092,61092
6,D2,281676,281676,281676
7,D3,281676,281676,281676
8,D4,281676,281676,281676
9,D5,281676,281676,281676


## Clean-up: Delete source tables

In [25]:
source_tables = {
    "person": person,
    "park": park,
    "team": team,
    "log": log}

with sqlite3.connect(db) as conn: 
    for name, data in tables.items():
        conn.execute('DROP TABLE IF EXISTS {};'.format(name))
        
show_tables()

,name,type
0,dim_person,table
1,dim_park,table
2,dim_league,table
3,appearance_type,table
4,dim_team,table
5,dim_game,table
6,team_appearance,table
7,person_appearance,table


## Schema Diagram

<img src="schema_diagram.png">
<img src="schema_diagram2.png">

## Clean-up: convert date

The dim_game table has a date field that is currently stored as text in the "YYYYMMDD" format. Converting to date string format YYYY-MM-DD for sqlite3.

In [36]:
c18 = """
UPDATE dim_game
SET date = SUBSTR(date, 1, 4) || "-" || SUBSTR(date, 5, 2)  || "-" || SUBSTR(date, 7, 2)
;"""

q10 = """SELECT date FROM dim_game LIMIT 10;"""

run_command(c18)
run_query(q10)

,date
0,1871-05-04
1,1871-05-05
2,1871-05-06
3,1871-05-08
4,1871-05-09
5,1871-05-11
6,1871-05-13
7,1871-05-13
8,1871-05-15
9,1871-05-16
